GWAS summery stat data preprocess
1.need to convert Markername to rsID by combining Chr and basepair coordinate in _lifted bim file and and replace Markername by matching
2.calculate z-score using P-value and effect size using munge_sumstat in ldsc github repository

In [1]:
#read meta5 file -- this is our GWAS summery stat data
! zless /data/CARD/PD/summary_stats/META5_all.gz

MarkerName      Allele1 Allele2 Freq1   FreqSE  MinFreq MaxFreq Effect  StdErr  P-value Direction       HetISq  HetChiSq        HetDf   HetPVal freqSpan
chr11:88249377  t       c       0.9905  0.0026  0.9882  0.9971  0.0238  0.0594  0.6889  -+?+++--+???-+++?       20      13.753  11      0.247   0.00890000000000002
chr1:60320992   a       g       0.9375  0.0049  0.9221  0.9473  0.0185  0.021   0.3801  +-?--+?++??-+---+       20.1    15.022  12      0.2402  0.0252
chr8:135908647  a       g       0.2112  0.007   0.1703  0.2316  -0.0054 0.0123  0.6598  --?+++?--??-----+       27.2    16.492  12      0.1697  0.0613
chr16:77148858  a       g       0.9963  5e-04   0.9961  0.9984  -0.0228 0.1356  0.8663  -??-+??+-???----?       0       4.331   8       0.8261  0.00229999999999997
chr11:97895884  c       g       0.0587  0.0035  0.0361  0.0685  -0.0219 0.0217  0.3148  +-?--+?--??+---++       0.2     12.029  12      0.4433  0.0324
chr2:80117945   a       g       0.9979  7e-04   0.9968  0.9988  -0

In [ ]:
#copy meta 5 file into data folder - run only once for download
! cp -R /data/CARD/PD/summary_stats/META5_all.gz /data/songy4/TWAS/data/

#unzip META5_all.gz - run only once for download
! gzip -d META5_all.gz

#make sumstat.csv which contains MarkerName, Allele1, Allele2, Effect, StdErr, P-value
! awk '{print $1, $2, $3, $8, $9, $10}' data_folder/META5_all > data_folder/sumstat.txt

In [1]:
#import library
import subprocess
import sys
import os
import shutil
import pandas as pd
import numpy as np

#set paths
basedir = '/data/songy4/twas'
datadir = f'{basedir}/data_folder'
fusiondir = f'{basedir}/fusion_twas'
geno_path = f'{datadir}/qc_genotypes_twas'
gene_list_path = f'{datadir}/gene_list.txt'
pheno_path = f'{datadir}/expression_matrix_final.txt'
coord_path = f'{datadir}/twas_coordinate.txt'
covar_path = f'{datadir}/covariates_case.txt'

In [2]:
statsum = pd.read_csv(r"./data_folder/META5_all", sep='\t')
statsum.head()

,MarkerName,Allele1,Allele2,Freq1,FreqSE,MinFreq,MaxFreq,Effect,StdErr,P-value,Direction,HetISq,HetChiSq,HetDf,HetPVal,freqSpan
0,chr11:88249377,t,c,0.9905,0.0026,0.9882,0.9971,0.0238,0.0594,0.6889,-+?+++--+???-+++?,20.0,13.753,11,0.2470,0.0089
1,chr1:60320992,a,g,0.9375,0.0049,0.9221,0.9473,0.0185,0.0210,0.3801,+-?--+?++??-+---+,20.1,15.022,12,0.2402,0.0252
2,chr8:135908647,a,g,0.2112,0.0070,0.1703,0.2316,-0.0054,0.0123,0.6598,--?+++?--??-----+,27.2,16.492,12,0.1697,0.0613
3,chr16:77148858,a,g,0.9963,0.0005,0.9961,0.9984,-0.0228,0.1356,0.8663,-??-+??+-???----?,0.0,4.331,8,0.8261,0.0023
4,chr11:97895884,c,g,0.0587,0.0035,0.0361,0.0685,-0.0219,0.0217,0.3148,+-?--+?--??+---++,0.2,12.029,12,0.4433,0.0324


In [3]:
#count if Freq1 is bigger than 0.5 --> this is to check if minor allele is A1
!cat {datadir}/META5_all | awk -F"\t" '$4>0.5' | wc -l

#check again how many Freq1 
statsum_flip = statsum[statsum.Freq1 > 0.5]
statsum_flip

4973527


,MarkerName,Allele1,Allele2,Freq1,FreqSE,MinFreq,MaxFreq,Effect,StdErr,P-value,Direction,HetISq,HetChiSq,HetDf,HetPVal,freqSpan
0,chr11:88249377,t,c,0.9905,0.0026,0.9882,0.9971,0.0238,0.0594,0.688900,-+?+++--+???-+++?,20.0,13.753,11,0.2470,0.0089
1,chr1:60320992,a,g,0.9375,0.0049,0.9221,0.9473,0.0185,0.0210,0.380100,+-?--+?++??-+---+,20.1,15.022,12,0.2402,0.0252
3,chr16:77148858,a,g,0.9963,0.0005,0.9961,0.9984,-0.0228,0.1356,0.866300,-??-+??+-???----?,0.0,4.331,8,0.8261,0.0023
5,chr2:80117945,a,g,0.9979,0.0007,0.9968,0.9988,-0.4366,0.4506,0.332600,?????+?--???---+?,0.0,5.236,6,0.5139,0.0020
6,chr18:814730,c,g,0.8893,0.0075,0.8639,0.9086,-0.0022,0.0176,0.902500,--?---?+-??+++--+,0.0,4.652,12,0.9686,0.0447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11477537,chr2:38455448,t,c,0.9929,0.0010,0.9867,0.9969,0.0027,0.0667,0.967600,-+?-++?-+?+?-++-?,0.0,6.288,11,0.8534,0.0102
11477538,chr7:65912486,t,g,0.9981,0.0005,0.9977,0.9988,0.8050,0.6315,0.202400,????+??+-????++??,0.0,2.030,4,0.7303,0.0011
11477540,chr18:50482478,c,g,0.9722,0.0023,0.9661,0.9807,-0.0203,0.0319,0.525300,--?-+-+-+??+++-+-,12.8,14.913,13,0.3128,0.0146
11477541,chr11:47906648,t,c,0.9452,0.0034,0.9326,0.9763,-0.0062,0.0207,0.765700,--+++++-+++--+-++,0.0,9.081,16,0.9101,0.0437


In [4]:
#flip Allele1 and Allele2 if Freq1 is bigger than .5 and change Freq1 to negative
stat_sum = statsum.copy()
cond = (stat_sum['Freq1'] > 0.5)
stat_sum.loc[cond, ['Allele1', 'Allele2']] = stat_sum.loc[cond, ['Allele2', 'Allele1']].values
#stat_sum_flip.Allele1, stat_sum_flip.Allele2 = np.where(stat_sum_flip.Freq1 > 0.5, [stat_sum_flip.Allele2, stat_sum_flip.Allele1], [stat_sum_flip.Allele1, stat_sum_flip.Allele2])

#change Freq1
stat_sum.loc[cond, ['Freq1']] = 1-stat_sum['Freq1']

#change Effect value
stat_sum.loc[cond, 'Effect'] *= -1
##no need to change  StdErr, P-value

#drop two rows Freq1 is smaller than 0.01
stat_sum.drop(stat_sum[stat_sum.Freq1 < 0.01].index, inplace=True)
stat_sum = stat_sum.reset_index(drop=True)
#stat_sum = stat_sum.drop(['index'], axis=1)

#drop unwanted columns
stat_sum.drop(stat_sum.iloc[:,10:16], axis=1, inplace=True)
stat_sum.drop(stat_sum.iloc[:,3:7], axis=1, inplace=True)

#check the unique number of MarkerName in meta
print("Unique MarkerName in meta data:", stat_sum['MarkerName'].nunique())

stat_sum

Unique MarkerName in meta data: 7784413


,MarkerName,Allele1,Allele2,Effect,StdErr,P-value
0,chr1:60320992,g,a,-0.0185,0.0210,0.380100
1,chr8:135908647,a,g,-0.0054,0.0123,0.659800
2,chr11:97895884,c,g,-0.0219,0.0217,0.314800
3,chr18:814730,g,c,0.0022,0.0176,0.902500
4,chr10:120407145,c,t,0.0278,0.0179,0.120300
...,...,...,...,...,...,...
7784408,chr18:50482478,g,c,0.0203,0.0319,0.525300
7784409,chr11:47906648,c,t,0.0062,0.0207,0.765700
7784410,chr3:47855594,c,t,-0.0348,0.0105,0.000944
7784411,chr3:194372962,a,g,0.0268,0.0500,0.591900


In [9]:
#save stat_sum 
stat_sum.to_csv(r'./data_folder/sumstat.txt', sep='\t' ,index=False)

In [1]:
def shell_do(command, log=False, return_log=False):
    print(f'Executing: {(" ").join(command.split())}', file=sys.stderr)

    res=subprocess.run(command.split(), stdout=subprocess.PIPE)

    if log:
        print(res.stdout.decode('utf-8'))
    if return_log:
        return(res.stdout.decode('utf-8'))

In [10]:
#open bim file 
bim_df = pd.read_csv(f"{geno_path}_hg19_lifted.bim", sep='\t', header=None,  names=('chr', 'rsid', 'kb', 'pos', 'a1', 'a2'))

#combine chr and pos columns
bim_df['MarkerName'] = str('chr') + bim_df.chr.astype(str) + ':' + bim_df.pos.astype(str)

#check the shape of meta
print("shape of bim data:", bim_df.shape)

#check the shape of meta
print("number of unique MarkerName in bim data:", bim_df['MarkerName'].nunique())
print("number of unique rsid in bim data:", bim_df['rsid'].nunique())
bim_df.head()

shape of bim data: (23060234, 7)
number of unique MarkerName in bim data: 23060188
number of unique rsid in bim data: 23060234


,chr,rsid,kb,pos,a1,a2,MarkerName
0,1,rs145427775,0,10291,T,C,chr1:10291
1,1,rs55998931,0,10492,T,C,chr1:10492
2,1,rs199896944,0,13504,A,G,chr1:13504
3,1,rs199856693,0,14933,A,G,chr1:14933
4,1,rs201855936,0,14948,A,G,chr1:14948


In [11]:
#print duplicated rows
bim_df_dup = bim_df[bim_df['MarkerName'].duplicated(keep=False)]
print('number of unique MarkerName in duplicated bim_df: ', bim_df_dup['MarkerName'].nunique())
bim_df_dup
## there are 46 duplicated MarkerName with 2 different rsid

number of unique MarkerName in duplicated bim_df:  46


,chr,rsid,kb,pos,a1,a2,MarkerName
978590,1,rs368379639,0,148609803,A,G,chr1:148609803
978719,1,rs372742293,0,148561459,T,C,chr1:148561459
979275,1,rs377761119,0,148609803,A,G,chr1:148609803
991861,1,rs111636993,0,148561459,T,C,chr1:148561459
5405594,3,rs368733146,0,195218551,T,G,chr3:195218551
...,...,...,...,...,...,...,...
22766969,22,rs188638512,0,19669438,T,C,chr22:19669438
22767437,22,rs374752842,0,19719099,T,C,chr22:19719099
22767608,22,rs41295659,0,19738319,A,C,chr22:19738319
22767625,22,rs41295687,0,19739438,A,G,chr22:19739438


In [12]:
print('Unique allele 1 in duplicated bim_df: ', bim_df_dup['a1'].unique())
print('Unique allele 2 in duplicated bim_df: ', bim_df_dup['a2'].unique())
#duplicated marker name
dup_marker = bim_df_dup['MarkerName'].unique().tolist()

#print gwas summery stat that has duplicated MarkerName
stat_sum_dup = stat_sum[stat_sum['MarkerName'].isin(dup_marker)]
stat_sum_dup

Unique allele 1 in duplicated bim_df:  ['A' 'T' 'C' '*' 'G' 'AAGAC']
Unique allele 2 in duplicated bim_df:  ['G' 'C' 'T' 'ACC' 'TG' 'A' 'AC' 'ACAG' 'CCTCCTGCCCAG']


,MarkerName,Allele1,Allele2,Effect,StdErr,P-value
3617568,chr22:19503090,c,t,-0.0449,0.0637,0.48050
3861458,chr22:19172321,t,g,0.0117,0.0183,0.52360
3939528,chr22:19497230,c,t,-0.0466,0.0263,0.07624
4745961,chr22:19390804,c,t,-0.0161,0.0164,0.32640
6030528,chr22:19643104,c,t,-0.0006,0.0134,0.96150


In [13]:
#print bim_df_dup that has duplicated MarkerName to see rsid
double_marker  = stat_sum_dup['MarkerName'].tolist()
bim_df_dup_exist = bim_df_dup[bim_df_dup['MarkerName'].isin(double_marker)]

#sort the dataframe by MarkerName
bim_df_dup_exist = bim_df_dup_exist.sort_values(by = 'MarkerName')

bim_df_dup_exist

,chr,rsid,kb,pos,a1,a2,MarkerName
22740222,22,rs199687134,0,19172321,C,T,chr22:19172321
22762532,22,rs1780641,0,19172321,T,G,chr22:19172321
22740918,22,rs141585725,0,19390804,T,C,chr22:19390804
22764321,22,rs11089274,0,19390804,C,T,chr22:19390804
22741203,22,rs2521701,0,19497230,T,C,chr22:19497230
22765096,22,rs117551893,0,19497230,C,T,chr22:19497230
22741234,22,rs370330113,0,19503090,C,T,chr22:19503090
22765143,22,rs138605059,0,19503090,C,T,chr22:19503090
22741527,22,rs145711952,0,19643104,A,G,chr22:19643104
22766688,22,rs9618649,0,19643104,C,T,chr22:19643104


In [14]:
#merge stat_sum with bim_df
meta_bim = pd.merge(stat_sum, bim_df, on=['MarkerName'], how='inner')
meta_bim

,MarkerName,Allele1,Allele2,Effect,StdErr,P-value,chr,rsid,kb,pos,a1,a2
0,chr1:60320992,g,a,-0.0185,0.0210,0.380100,1,rs116406626,0,60320992,G,A
1,chr8:135908647,a,g,-0.0054,0.0123,0.659800,8,rs11992603,0,135908647,A,G
2,chr10:120407145,c,t,0.0278,0.0179,0.120300,10,rs4336949,0,120407145,C,T
3,chr11:29372878,t,c,0.0049,0.0218,0.822100,11,rs72638174,0,29372878,T,C
4,chr3:164053059,a,g,0.0607,0.0488,0.214100,3,rs115079612,0,164053059,A,G
...,...,...,...,...,...,...,...,...,...,...,...,...
6478463,chr16:84525085,t,c,-0.0387,0.0310,0.211700,16,rs2927754,0,84525085,T,C
6478464,chr11:47906648,c,t,0.0062,0.0207,0.765700,11,rs148423893,0,47906648,C,T
6478465,chr3:47855594,c,t,-0.0348,0.0105,0.000944,3,rs62263573,0,47855594,C,T
6478466,chr3:194372962,a,g,0.0268,0.0500,0.591900,3,rs149847890,0,194372962,A,G


In [15]:
#check duplicated rsid
meta_bim_dup = meta_bim[meta_bim['rsid'].duplicated(keep=False)]
print('number of unique rsid in duplicated meta_bim: ', meta_bim_dup['rsid'].nunique())
#check number of unique rsid in case
print('number of unique rsid in meta_bim: ', meta_bim.rsid.nunique()) 
#check number of unique MarkerName in case
print('number of unique MarkerName in meta_bim: ', meta_bim.MarkerName.nunique())

#print gwas summery stat with SNP that has duplicated MarkerName
snp_stat_dup = meta_bim[meta_bim['MarkerName'].isin(dup_marker)]
snp_stat_dup
#7 duplicated (MarkerName) position with different rsid

number of unique rsid in duplicated meta_bim:  0
number of unique rsid in meta_bim:  6478468
number of unique MarkerName in meta_bim:  6478463


,MarkerName,Allele1,Allele2,Effect,StdErr,P-value,chr,rsid,kb,pos,a1,a2
3011341,chr22:19503090,c,t,-0.0449,0.0637,0.48050,22,rs370330113,0,19503090,C,T
3011342,chr22:19503090,c,t,-0.0449,0.0637,0.48050,22,rs138605059,0,19503090,C,T
3214210,chr22:19172321,t,g,0.0117,0.0183,0.52360,22,rs199687134,0,19172321,C,T
3214211,chr22:19172321,t,g,0.0117,0.0183,0.52360,22,rs1780641,0,19172321,T,G
3279286,chr22:19497230,c,t,-0.0466,0.0263,0.07624,22,rs2521701,0,19497230,T,C
3279287,chr22:19497230,c,t,-0.0466,0.0263,0.07624,22,rs117551893,0,19497230,C,T
3950528,chr22:19390804,c,t,-0.0161,0.0164,0.32640,22,rs141585725,0,19390804,T,C
3950529,chr22:19390804,c,t,-0.0161,0.0164,0.32640,22,rs11089274,0,19390804,C,T
5019123,chr22:19643104,c,t,-0.0006,0.0134,0.96150,22,rs145711952,0,19643104,A,G
5019124,chr22:19643104,c,t,-0.0006,0.0134,0.96150,22,rs9618649,0,19643104,C,T


In [16]:
#drop rows in meta if rsid is NaN 
#meta_df = snp_stat[snp_stat['SNP'].notna()]

sumstat_df = meta_bim.copy()
#change Allele1 and Allele2 to A1 and A2
sumstat_df = sumstat_df.rename(columns={'Allele1':'A1', 'Allele2':'A2','rsid':'SNP'})
sumstat_df = sumstat_df.drop(['a1', 'a2', 'kb'], axis=1)

#reset index
#meta_df = meta_df.reset_index(drop=True)

#check numbers of rows in meta_df
print("numbers of rows in meta data:", sumstat_df.shape[0])

sumstat_df.head()

numbers of rows in meta data: 6478468


,MarkerName,A1,A2,Effect,StdErr,P-value,chr,SNP,pos
0,chr1:60320992,g,a,-0.0185,0.0210,0.3801,1,rs116406626,60320992
1,chr8:135908647,a,g,-0.0054,0.0123,0.6598,8,rs11992603,135908647
2,chr10:120407145,c,t,0.0278,0.0179,0.1203,10,rs4336949,120407145
3,chr11:29372878,t,c,0.0049,0.0218,0.8221,11,rs72638174,29372878
4,chr3:164053059,a,g,0.0607,0.0488,0.2141,3,rs115079612,164053059


In [17]:
#calculate z score with effect size and standard error (z = (score-mean)/se)

def z_score(a, b):
    if b == 0:
        return 0.0
    return float(a)/b

sumstat_df['Z'] = np.vectorize(z_score)(sumstat_df['Effect'], sumstat_df['StdErr'])

sumstat_df

,MarkerName,A1,A2,Effect,StdErr,P-value,chr,SNP,pos,Z
0,chr1:60320992,g,a,-0.0185,0.0210,0.380100,1,rs116406626,60320992,-0.880952
1,chr8:135908647,a,g,-0.0054,0.0123,0.659800,8,rs11992603,135908647,-0.439024
2,chr10:120407145,c,t,0.0278,0.0179,0.120300,10,rs4336949,120407145,1.553073
3,chr11:29372878,t,c,0.0049,0.0218,0.822100,11,rs72638174,29372878,0.224771
4,chr3:164053059,a,g,0.0607,0.0488,0.214100,3,rs115079612,164053059,1.243852
...,...,...,...,...,...,...,...,...,...,...
6478463,chr16:84525085,t,c,-0.0387,0.0310,0.211700,16,rs2927754,84525085,-1.248387
6478464,chr11:47906648,c,t,0.0062,0.0207,0.765700,11,rs148423893,47906648,0.299517
6478465,chr3:47855594,c,t,-0.0348,0.0105,0.000944,3,rs62263573,47855594,-3.314286
6478466,chr3:194372962,a,g,0.0268,0.0500,0.591900,3,rs149847890,194372962,0.536000


In [18]:
#function to move columns
def movecol(df, cols_to_move=[], ref_col='', place='After'):
    
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

#rearrange Gene_Symbol, Chr, Start, Stop columns right nextg to index column 
sumstat = movecol(sumstat_df, cols_to_move=['SNP'], ref_col='MarkerName', place='After')

#drop MarkerName, Effect, StdErr, P-value colunms 
sumstat = sumstat.drop(['MarkerName', 'Effect', 'StdErr', 'P-value', 'chr', 'pos'], axis=1)

sumstat = sumstat.reset_index(drop=True)
print("shape of meta data:", sumstat.shape)
sumstat.head()

##sumstat is ready to use

shape of meta data: (6478468, 4)


,SNP,A1,A2,Z
0,rs116406626,g,a,-0.880952
1,rs11992603,a,g,-0.439024
2,rs4336949,c,t,1.553073
3,rs72638174,t,c,0.224771
4,rs115079612,a,g,1.243852


In [21]:
#calculate chisq
stat_sum_chi = sumstat_df.copy()
stat_sum_chi['chisq'] = stat_sum_chi['Z']**2
print(stat_sum_chi)

#calculate lambda (median(z^2) / 0.455)
lambda_stat_sum = ((stat_sum_chi['chisq'].median())/0.455)
print("lambda using chisq: ", lambda_stat_sum)

#calculate lambda (median(z^2) / 0.455)
lambda_statsum = ((stat_sum_chi['P-value'].median())/0.456)
print("lambda using p-value: ", lambda_statsum)

              MarkerName A1 A2  Effect  StdErr   P-value  chr          SNP  \
0          chr1:60320992  g  a -0.0185  0.0210  0.380100    1  rs116406626   
1         chr8:135908647  a  g -0.0054  0.0123  0.659800    8   rs11992603   
2        chr10:120407145  c  t  0.0278  0.0179  0.120300   10    rs4336949   
3         chr11:29372878  t  c  0.0049  0.0218  0.822100   11   rs72638174   
4         chr3:164053059  a  g  0.0607  0.0488  0.214100    3  rs115079612   
...                  ... .. ..     ...     ...       ...  ...          ...   
6478463   chr16:84525085  t  c -0.0387  0.0310  0.211700   16    rs2927754   
6478464   chr11:47906648  c  t  0.0062  0.0207  0.765700   11  rs148423893   
6478465    chr3:47855594  c  t -0.0348  0.0105  0.000944    3   rs62263573   
6478466   chr3:194372962  a  g  0.0268  0.0500  0.591900    3  rs149847890   
6478467   chr19:34986930  a  c  0.0209  0.0154  0.174900   19   rs13353264   

               pos         Z      chisq  
0         60320992 -0

In [21]:
#save meta_df as txt -- run only once for download
sumstat.to_csv(r'./data_folder/meta.txt', sep='\t' ,index=False)
#gwas sumary stat is ready to use

Run this after post processing

In [22]:
##grab significant GWAS hits (p <= 5e-8)
sig_meta = sumstat_df[sumstat_df['P-value']< 5e-8]
#reset index
sig_meta = sig_meta.reset_index(drop=True)
print("shape of significant GWAS data:", sig_meta.shape)

sig_meta.head()

shape of significant GWAS data: (5891, 10)


,MarkerName,A1,A2,Effect,StdErr,P-value,chr,SNP,pos,Z
0,chr16:31054607,t,c,-0.0831,0.0099,6.178000e-17,16,rs1108431,31054607,-8.393939
1,chr10:121534732,t,c,0.0587,0.0095,5.506000e-10,10,rs196202,121534732,6.178947
2,chr5:102446462,a,c,-0.0603,0.0103,4.716000e-09,5,rs257320,102446462,-5.854369
3,chr16:31023273,a,g,-0.0852,0.0100,1.338000e-17,16,rs12447930,31023273,-8.520000
4,chr4:951040,g,t,-0.1074,0.0097,1.054000e-28,4,rs3733344,951040,-11.072165


In [33]:
sig_meta.pos.dtype

dtype('int64')

In [24]:
#check if double marker are in sig_meta
sig_meta_dup = sig_meta[sig_meta['MarkerName'].isin(double_marker)]
sig_meta_dup

,MarkerName,A1,A2,Effect,StdErr,P-value,SNP,Z


In [23]:
#save sig_meta -- run only once
sig_meta.to_csv(r'./data_folder/gwas_pos.csv', index=False)

In [5]:
#bring in coordinate data
coords = pd.read_csv(coord_path, sep='\t')
coords

,X.Chr,start,end,ID
0,1,69091,70008,ENSG00000186092
1,1,860260,879955,ENSG00000187634
2,1,879584,894689,ENSG00000188976
3,1,895967,901095,ENSG00000187961
4,1,901877,911245,ENSG00000187583
...,...,...,...,...
18236,21,47720095,47743789,ENSG00000160298
18237,21,47744036,47865682,ENSG00000160299
18238,21,47878812,47989926,ENSG00000160305
18239,21,48018875,48025121,ENSG00000160307


In [25]:
#create pos_less and pos_more which are position plus minus 1 megabase
sig_meta['pos_less']= sig_meta['pos'] - 500000
sig_meta['pos_more']= sig_meta['pos'] + 500000
sig_meta

,MarkerName,A1,A2,Effect,StdErr,P-value,chr,SNP,pos,Z,pos_less,pos_more
0,chr16:31054607,t,c,-0.0831,0.0099,6.178000e-17,16,rs1108431,31054607,-8.393939,30554607,31554607
1,chr10:121534732,t,c,0.0587,0.0095,5.506000e-10,10,rs196202,121534732,6.178947,121034732,122034732
2,chr5:102446462,a,c,-0.0603,0.0103,4.716000e-09,5,rs257320,102446462,-5.854369,101946462,102946462
3,chr16:31023273,a,g,-0.0852,0.0100,1.338000e-17,16,rs12447930,31023273,-8.520000,30523273,31523273
4,chr4:951040,g,t,-0.1074,0.0097,1.054000e-28,4,rs3733344,951040,-11.072165,451040,1451040
...,...,...,...,...,...,...,...,...,...,...,...,...
5886,chr3:151114889,g,a,-0.0618,0.0098,2.501000e-10,3,rs17204501,151114889,-6.306122,150614889,151614889
5887,chr6:32587090,t,c,-0.0833,0.0125,2.978000e-11,6,rs28383312,32587090,-6.664000,32087090,33087090
5888,chr14:56192450,c,t,-0.0557,0.0094,2.820000e-09,14,rs7157327,56192450,-5.925532,55692450,56692450
5889,chr8:16735866,t,c,-0.0850,0.0123,4.934000e-12,8,rs35847654,16735866,-6.910569,16235866,17235866


Read the 90 parkinson's GWAS hits from Mike's paper (some are in the supplement) ##https://www.thelancet.com/journals/laneur/article/PIIS1474-4422(19)30320-5/fulltext

In [ ]:
#read 90 parkinson's GWAS hits
pd_hits = pd.read_csv(r"./data_folder/PD_GRS_dbSNPs.tab", sep='\t', lineterminator='\r', header=None,  names=('rsid', 'author', 'condition', 'name') )
pd_hits.drop(pd_hits.iloc[:,1:4], axis=1, inplace=True)
pd_hits['rsid'] = pd_hits['rsid'].str.replace('\n', '')
pd_hits['rsid'].replace('', np.nan, inplace=True)
pd_hits.dropna(subset=['rsid'], inplace=True)
pd_hits

In [ ]:
#merge rsid with pos from meta data

pd_hits_pos_1 = pd.merge(bim_df, pd_hits, on=['rsid'], how='inner')
pd_hits_pos_1

In [ ]:
#get the rsid without pos
mis_rsid = pd_hits[~pd_hits['rsid'].isin(pd_hits_pos_1['rsid'])]
mis_rsid = mis_rsid.reset_index(drop=True)
mis_rsid

In [ ]:
#merge rsid with pos from meta data

pd_hits_pos_2 = pd_hits.merge(bim_df, on=['rsid'], how='left') 
pd_hits_pos_2 = pd_hits_pos_2.drop(['kb'], axis=1)
pd_hits_pos_2

In [ ]:
#input data for rs114138760
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs114138760', 'chr'] = '1'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs114138760', 'pos'] = '154898185'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs114138760', 'a1'] = 'C'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs114138760', 'a2'] = 'G'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs114138760', 'MarkerName'] = 'chr1:154898185'

#input data for rs6658353
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6658353', 'chr'] = '1'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6658353', 'pos'] = '161469054'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6658353', 'a1'] = 'C'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6658353', 'a2'] = 'G'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6658353', 'MarkerName'] = 'chr1:161469054'

#input data for rs76116224
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs76116224', 'chr'] = '2'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs76116224', 'pos'] = '18147848'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs76116224', 'a1'] = 'T'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs76116224', 'a2'] = 'A'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs76116224', 'MarkerName'] = 'chr2:18147848'

#input data for rs2042477
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs2042477', 'chr'] = '2'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs2042477', 'pos'] = '96000943'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs2042477', 'a1'] = 'T'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs2042477', 'a2'] = 'A'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs2042477', 'MarkerName'] = 'chr2:96000943'

#input data for rs11683001
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs11683001', 'chr'] = '2'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs11683001', 'pos'] = '102396963'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs11683001', 'a1'] = 'A'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs11683001', 'a2'] = 'T'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs11683001', 'MarkerName'] = 'chr2:102396963'

#input data for rs11707416
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs11707416', 'chr'] = '3'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs11707416', 'pos'] = '151108965'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs11707416', 'a1'] = 'A'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs11707416', 'a2'] = 'T'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs11707416', 'MarkerName'] = 'chr3:151108965'

#input data for rs34025766
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs34025766', 'chr'] = '4'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs34025766', 'pos'] = '17968811'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs34025766', 'a1'] = 'A'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs34025766', 'a2'] = 'T'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs34025766', 'MarkerName'] = 'chr4:17968811'

#input data for rs6825004
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6825004', 'chr'] = '4'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6825004', 'pos'] = '77110365'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6825004', 'a1'] = 'G'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6825004', 'a2'] = 'C'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6825004', 'MarkerName'] = 'chr4:77110365'

#input data for rs26431
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs26431', 'chr'] = '5'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs26431', 'pos'] = '102365794'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs26431', 'a1'] = 'C'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs26431', 'a2'] = 'G'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs26431', 'MarkerName'] = 'chr5:102365794'

#input data for rs76949143
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs76949143', 'chr'] = '7'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs76949143', 'pos'] = '66009851'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs76949143', 'a1'] = 'A'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs76949143', 'a2'] = 'T'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs76949143', 'MarkerName'] = 'chr7:66009851'

#input data for rs6476434
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6476434', 'chr'] = '9'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6476434', 'pos'] = '34046391'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6476434', 'a1'] = 'T'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6476434', 'a2'] = 'C'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6476434', 'MarkerName'] = 'chr9:34046391'

#input data for rs6497339
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6497339', 'chr'] = '16'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6497339', 'pos'] = '19277493'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6497339', 'a1'] = 'T'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6497339', 'a2'] = 'A'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6497339', 'MarkerName'] = 'chr16:19277493'

#input data for rs2904880
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs2904880', 'chr'] = '16'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs2904880', 'pos'] = '28944396'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs2904880', 'a1'] = 'G'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs2904880', 'a2'] = 'C'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs2904880', 'MarkerName'] = 'chr16:28944396'

#input data for rs666463
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs666463', 'chr'] = '17'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs666463', 'pos'] = '76425480'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs666463', 'a1'] = 'T'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs666463', 'a2'] = 'A'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs666463', 'MarkerName'] = 'chr17:76425480'

#convert column values to int
col_case=pd_hits_pos_2.columns[1:3]
pd_hits_pos_2.loc[:,col_case]=pd_hits_pos_2.loc[:,col_case].astype(np.int64)

pd_hits_pos_2

In [ ]:
#save pd_hits_pos_2 
pd_hits_pos_2.to_csv(r'./data_folder/PD_GRS_dbSNPs_position.csv', index=False)

In [ ]:
#create pos_less and pos_more which are position plus minus 1 megabase
pd_hits_pos_2['pos_less']= pd_hits_pos_2['pos'] - 1000000
pd_hits_pos_2['pos_more']= pd_hits_pos_2['pos'] + 1000000
pd_hits_pos_2
#read 90 parkinson's GWAS hits
#pd_hits_pos_2 = pd.read_csv(r"./data_folder/PD_GRS_dbSNPs_position.csv")
#pd_hits_pos_2

In [6]:
#grab gene ID from coords file if position is within start and end in coords ---> coords['start'] <= row['pos'] <= coords['end']

from numpy import nan

def match_labels(row):
    curr_df = coords[(coords['X.Chr'] == row['chr']) & ((coords['start'] <= row['pos']) & (coords['end'] >= row['pos']))]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.NaN

    return row

result_1 = pd_hits_pos_2.apply(lambda x:match_labels(x),axis=1)

result_1

#grab gene ID from coords file if position is within start and end in coords ---> coords['start']<= row['pos_more'] <= coords['end'] or coords['end'] >= row['pos_less'] >= coords['start']

from numpy import nan

def match_labels(row):
    curr_df = coords[(coords['X.Chr'] == row['chr']) & (((coords['start'] <= row['pos_more']) & (coords['end']) >= row['pos_more']) | ((coords['start'] <= row['pos_less']) & (coords['end'] >= row['pos_less'])))]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.NaN

    return row

result_2 = pd_hits_pos_2.apply(lambda x:match_labels(x),axis=1)

result_2

#grab gene ID from coords file if position is within start and end in coords ---> row['pos_less'] <= coords['start'] <= row['pos_more'] or row['pos_less'] <= coords['end'] <= row['pos_more']

from numpy import nan

def match_labels(row):
    curr_df = coords[(coords['X.Chr'] == row['chr']) & (((coords['start'] <= row['pos_more']) & (coords['start'] >= row['pos_less'])) | ((coords['end'] >= row['pos_less']) & (coords['end'] <= row['pos_more'])))]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.NaN

    return row

result_3 = pd_hits_pos_2.apply(lambda x:match_labels(x),axis=1)

result_3

###row['pos_less'] <= coords['start'] <= row['pos_more'] or row['pos_less'] <= coords['end'] <= row['pos_more']

,rsid,chr,pos,a1,a2,MarkerName,pos_less,pos_more,ID
0,rs114138760,1,154898185,C,G,chr1:154898185,153898185,155898185,ENSG00000163344
1,rs35749011,1,155135036,A,G,chr1:155135036,154135036,156135036,NaN
2,rs76763715,1,155205634,C,T,chr1:155205634,154205634,156205634,ENSG00000177628
3,rs6658353,1,161469054,C,G,chr1:161469054,160469054,162469054,NaN
4,rs11578699,1,171719769,T,C,chr1:171719769,170719769,172719769,NaN
...,...,...,...,...,...,...,...,...,...
85,rs12456492,18,40673380,G,A,chr18:40673380,39673380,41673380,ENSG00000152214
86,rs8087969,18,48683589,G,T,chr18:48683589,47683589,49683589,NaN
87,rs55818311,19,2341047,C,T,chr19:2341047,1341047,3341047,ENSG00000005206
88,rs77351827,20,6006041,T,C,chr20:6006041,5006041,7006041,ENSG00000088766


In [7]:
print('Null ID in reult_1: ',result_1.ID.isnull().sum())

#get non null values
print('Non Null ID in reult_1: ',result_1[['ID']].T.stack().tolist())
#result_1[['ID']].T.stack().values

print('Null ID in reult_2: ',result_2.ID.isnull().sum())

#get non null values
print('Non Null ID in reult_2: ',result_2[['ID']].T.stack().tolist())
#result_1[['ID']].T.stack().values

print('Null ID in reult_3: ',result_3.ID.isnull().sum())

#get non null values
print('Non Null ID in reult_3: ',result_3[['ID']].T.stack().tolist())
#result_1[['ID']].T.stack().values

Null ID in reult_1:  25
Non Null ID in reult_1:  ['ENSG00000163344', 'ENSG00000177628', 'ENSG00000117280', 'ENSG00000143772', 'ENSG00000116991', 'ENSG00000170745', 'ENSG00000115041', 'ENSG00000071054', 'ENSG00000152128', 'ENSG00000131374', 'ENSG00000068745', 'ENSG00000114030', 'ENSG00000144893', 'ENSG00000196542', 'ENSG00000078070', 'ENSG00000178950', 'ENSG00000127419', 'ENSG00000109743', 'ENSG00000178177', 'ENSG00000138760', 'ENSG00000189157', 'ENSG00000189157', 'ENSG00000109572', 'ENSG00000164181', 'ENSG00000145730', 'ENSG00000204614', 'ENSG00000136235', 'ENSG00000164733', 'ENSG00000147439', 'ENSG00000153310', 'ENSG00000107295', 'ENSG00000107295', 'ENSG00000137073', 'ENSG00000077943', 'ENSG00000107862', 'ENSG00000151929', 'ENSG00000198825', 'ENSG00000110315', 'ENSG00000150672', 'ENSG00000080854', 'ENSG00000188906', 'ENSG00000188906', 'ENSG00000130787', 'ENSG00000102547', 'ENSG00000151338', 'ENSG00000131979', 'ENSG00000198208', 'ENSG00000103528', 'ENSG00000177548', 'ENSG00000099381', 

In [24]:
#drop all columns except ID
gwas_genes_1 = result_1.copy()
gwas_genes_1.drop(gwas_genes_1.iloc[:,0:8], axis=1, inplace=True)
gwas_genes_1.dropna(inplace=True)
gwas_genes_1 = gwas_genes_1.reset_index(drop=True)

gwas_genes_2 = result_2.copy()
gwas_genes_2.drop(gwas_genes_2.iloc[:,0:8], axis=1, inplace=True)
gwas_genes_2.dropna(inplace=True)
gwas_genes_2 = gwas_genes_2.reset_index(drop=True)

gwas_genes_3 = result_3.copy()
gwas_genes_3.drop(gwas_genes_3.iloc[:,0:8], axis=1, inplace=True)

In [21]:
#check common IDs between result_1~3
gene_1 = gwas_genes_1.ID.tolist()
gene_2 = gwas_genes_2.ID.tolist()
gene_3 = result_3.ID.tolist()

print('number of ID in result_1: ', len(gene_1))
print('number of ID in result_2: ', len(gene_2))
print('number of ID in result_3: ', len(gene_3))

number of ID in result_1:  65
number of ID in result_2:  40
number of ID in result_3:  90


In [22]:
def comprehension(a, b):
    return [x for x in a if x in b]

list_1 = comprehension(gene_1,gene_2)
list_2 = comprehension(gene_1,gene_3)
list_3 = comprehension(gene_2,gene_3)

print('number of ID in result_1 and result_2: ', len(list_1))
print('number of ID in result_1 and result_3: ', len(list_2))
print('number of ID in result_2 and result_3: ', len(list_3))
#since all of gene_2 are in gene_3 we will combine gene_1 and gene_3

number of ID in result_1 and result_2:  3
number of ID in result_1 and result_3:  4
number of ID in result_2 and result_3:  40


In [31]:
#merge gwas_genes_1 and gwas_genes_3
gwas = pd.merge(gwas_genes_1, gwas_genes_3, on='ID', how='outer')
gwas = gwas.drop_duplicates(['ID']).reset_index(drop=True)
gwas.shape

(142, 1)

In [32]:
#save gwas
gwas.to_csv(r'./data_folder/PD_genes.csv', index=False)